---
 
title: "Python for Hardware Engineers: SQLite or; How I Learned to Stop Worrying and Love Databases"
subtitle: ""
summary: ""
authors: []
tags: []
categories: []
date: 2020-11-10T21:35:08-05:00
lastmod: 2020-11-10T21:35:08-05:00
featured: false
draft: false
image:
  caption: ""
  focal_point: ""
  preview_only: false
projects: []
 
---

All my career, I've worked at the intersection of hardware and software.
Data collection, processing, and analysis are fundamental skills that engineers need to create effective solutions and data-driven designs.
A big reason for this [Python for Hardware Engineers]({{< ref "/post/2020/10/new-series-python-for-hardware-engineering" >}}) series is because I believe that the standard educational pathways that hardware engineers follow isn't sufficient for modern data-driven practices.

In university, our mechanical and electrical engineering lab sessions may require us to collect a few dozen data recordings and produce a report.
Even our [semi-realistic temperature and humidity data processing example using a real data logger]({{< ref "/post/2020/10/python-for-hardware-engineers-analyzing-temperature-sensor-data" >}}) only collected recordings at 30min intervals.
This would only produce a few hundred recordings during a week of constant data collection, something which Excel or simple `pandas` scripts can easily handle.

But what happens when we collect more data?
At what point does data get "big" for a hardware engineer?
When do I need a fancy database?

Well, I'll let you in on a little secret: 95% of the time, recording data to disk is perfectly fine for what we do.
It's just that last 5% of the time where we wished we had used a database...

## Saving Data to Disk

There's nothing I love more than setting up an experiment, clicking go, and watching my PC monitor the progress.
For example, when I developed a [closed-loop robot calibration technique]({{< ref "publication/nadeau-2019-impedance" >}}), I sat back and watched my collaborative robot dance around for a while as my laptop recorded each robot pose and laser tracker measure.

![Robot Calibration](IMG_7420.jpg)

How much data was collected to successfully calibrate a robot's absolute accuracy?
Just 1875 measures. That's it. No need for a database here.
As a matter of fact, I designed the recording function to dump each recording straight to disk as a JSON file, such that nothing was kept in memory for too long just in case an error occurred (read: robot collision).

So here I present the most important data collection functions for a hardware engineer:

In [53]:
import json
from pathlib import Path
import datetime

DATA_DIR = Path() / 'output'
DATA_DIR.mkdir(exist_ok=True)

def record_data() -> dict:
    """Fetch the data and return it as a Python dict"""
    data = {}
    # get data from somewhere, e.g.:
    data['temperature'] = 123
    data['humidity'] = 456
    return data

def save_data(data: dict, dir_path:Path) -> None:
    """Save the data to an output folder"""
    now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S_%f")
    path = dir_path / f"{now}_recording.json"
    with open(path,'w') as f:
        json.dump(data, f)
        
# get data
d = record_data()

# dump to disk
save_data(d, DATA_DIR)

The above functions will serve you perfectly well up to a few thousand recordings.
Therefore, my general philosophy for when to save data straight to disk:

- When I have unstructured data in reasonable amounts that the file system can handle
- When I don't want or can't deal with the overhead a database will add

## When Data Gets Too Big

At [AON3D](https://www.aon3d.com/) we have a fleet of printers used internally for customer success, engineering, R&D, and sales.
As an internal experiment and side-project, I like to record all our internal printers' temperatures (two toolhead sensors, one chamber sensor, and one bed sensor) over time to have a better understanding of system reliability and prototype "hot-time" during testing. 
Each printer reports its temperature at a rate of 1Hz.
I record 2s of data, per printer, every 5min to have a high-level overview of our internal fleet.
Each week I produce over 40K recordings totaling several hundred megabytes of data.

Initially, I was lazy and dumped the data to disk using the method described above.
Each week I ran a script that consolidates the data to a CSV, such that other stakeholders can have easy access and weekly reports.
The consolidation process took a good 10-20min as each JSON file is loaded into memory, parsed into a `pandas` Series, concatenated into a DataFrame, and then exported. 

This is when data starts to get too big for saving to disk.

## Meet SQLite and `peewee`

SQLite is a library that provides a lightweight disk-based database that doesn’t require a separate server process.
It's the next step after dumping to disk and allows for future growth as you can one day port the code to a larger database, like PostgreSQL.
As a super basic overview of SQL:

- A database can have multiple tables (think Excel workbook having multiple sheets/tabs)
- Each table stores a set of data in rows and columns
- Each column is a data field and has a specific type (e.g., `int`, `float`, `str`, `datetime`, `bool`)
- Each row is an instance of data across all columns (i.e., a recording)

Fortunately, SQLite is built-in to Python, but using it directly is not necessarily too intuitive or "Pythonic".
As a solution, I recommend [`peewee`](https://github.com/coleifer/peewee/), a simple and small [object-relational-mapping (ORM)](https://en.wikipedia.org/wiki/Object%E2%80%93relational_mapping).
It has few expressive concepts to make it easy to learn and intuitive to use as a "Pythonic" wrapper around SQLite:

|`peewee` Object|Corresponds to…|
|---|---|
|Model class|Database table|
|Field instance|Column on a table|
|Model instance|Row in a database table|

Let's create some fake data and take a look:

In [2]:
from peewee import Model, CharField,ForeignKeyField,DateTimeField,FloatField, SqliteDatabase
import datetime

# we're using `:memory:` for this example,
# but you can use a filename (e.g., `my-database.db`) 
# if you want to save the database to disk
db = SqliteDatabase(':memory:')

class BaseModel(Model):
    """Base class that we can share across various data models"""
    class Meta:
        database = db

class Recording(BaseModel):
    """Fake data model for temperature/humidity measurement"""
    # datetime of recording
    # the `default` argument will auto-fill this field for us!
    timestamp = DateTimeField(default=datetime.datetime.now)
    
    # standard numeric data fields
    temperature = FloatField()
    humidity = FloatField()

# test the database connection
db.connect()

# create the db tables
db.create_tables([Recording])

In [3]:
# create a recording instance, i.e., a data row
recording = Recording.create(temperature=25, humidity=60)

# we can use the model instance like an object
print(f"Temp: {recording.temperature}")
print(f"RH: {recording.humidity}")# or create data from a good-old dict
d = {'temperature':30, 'humidity':80}
recording = Recording.create(**d)

Temp: 25
RH: 60


In [4]:
# or create data from a good-old dict
d = {'temperature':30, 'humidity':80}
recording = Recording.create(**d)

In [5]:
# import included helper functions
from playhouse.shortcuts import model_to_dict

# convert a row/instance back to dict!
model_to_dict(recording)

{'id': 3,
 'timestamp': datetime.datetime(2020, 11, 14, 18, 4, 8, 352596),
 'temperature': 30,
 'humidity': 80}

In [6]:
# convert the whole table back to dicts
list(Recording.select().dicts())

[{'id': 1,
  'timestamp': datetime.datetime(2020, 11, 14, 18, 4, 8, 341343),
  'temperature': 25.0,
  'humidity': 60.0},
 {'id': 2,
  'timestamp': datetime.datetime(2020, 11, 14, 18, 4, 8, 342107),
  'temperature': 30.0,
  'humidity': 80.0},
 {'id': 3,
  'timestamp': datetime.datetime(2020, 11, 14, 18, 4, 8, 352596),
  'temperature': 30.0,
  'humidity': 80.0}]

In [27]:
import pandas as pd

# convert the whole table to a dataframe!
pd.DataFrame(Recording.select().dicts())

,id,timestamp,temperature,humidity
0,1,2020-11-14 18:04:08.341343,25.0,60.0
1,2,2020-11-14 18:04:08.342107,30.0,80.0
2,3,2020-11-14 18:04:08.352596,30.0,80.0
3,4,2020-11-14 18:05:40.267193,1.0,2.0
4,5,2020-11-14 18:05:41.004610,1.0,2.0
...,...,...,...,...
243667,243668,2020-11-14 18:11:29.276440,30.0,80.0
243668,243669,2020-11-14 18:11:29.276553,30.0,80.0
243669,243670,2020-11-14 18:11:29.276670,30.0,80.0
243670,243671,2020-11-14 18:11:29.276784,30.0,80.0


## Did Someone Say Performance?

In [22]:
d = {'temperature':30, 'humidity':80}

In [21]:
%timeit Recording.create(temperature=30,humidity=80)

117 µs ± 4.96 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [20]:
%timeit Recording.create(**d)

124 µs ± 5.27 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [54]:
%timeit save_data(d,dir_path=DATA_DIR)

89.9 µs ± 6.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Getting a Single Recording

In [29]:
# get first record with condition
Recording.select().where(Recording.temperature == 30).get()

<Recording: 2>

In [55]:
paths = list(DATA_DIR.glob('*.json'))
print(f"Found {len(paths)} paths")

Found 81127 paths


In [56]:
# consolidate many JSONs to a dataframe for CSV export...
data = map(lambda x: pd.read_json(x, typ='series'), paths)
df = pd.DataFrame(data)

KeyboardInterrupt: 